# Unified DGLAP

In [3]:
import sympy

In [5]:
# QCD
Pv, Pp, Pm, Pqq, Pqg, Pgq, Pgg = sympy.symbols("P_V P_+ P_- P_qq P_qg P_gq P_gg")
# QED
Pxv, Pxp, Pxm, Pxqq, Pxqg, Pxgq, Pxgg = sympy.symbols("P^x_V P^x_+ P^x_- P^x_qq P^x_qg P^x_gq P^x_gg")
Pxqy, Pxyq, Pxyg, Pxgy, Pxyy = sympy.symbols("P^x_q\gamma P^x_\gamma\ q P^x_\gamma\ g P^x_g\gamma P^x_\gamma\gamma")
eu2, ed2, es2 = sympy.symbols("e_u^2 e_d^2 e_\Sigma^2") # charges

## Flavor basis :
### [g, \gamma, u+, u-, d+, d-, s+, s-, c+, c-, b+, b-, t+, t-]
## Singlet basis :
### [g, \gamma, \Sigma_u, \Sigma_d, V_u, V_d, T_3^d, V_3^d, T_3^u / c+, V_3^u / c-, T_8^d / b+, V_8^d / b-, T_8^u / t+, V_8^u / t-]
## Intrinsic evolution basis :
### [g, \gamma, \Sigma, \Delta_\Sigma, V, \Delta_V, T_3^d, V_3^d, T_3^u / c+, V_3^u / c-, T_8^d / b+, V_8^d / b-, T_8^u / t+, V_8^u / t-]
### [T_3^u, V_3^u, T_8^d, V_8^d, T_8^u, V_8^u] is:
### [c+, c-, b+, b-, t+, t-] in nf=3,
### [T_3^u, V_3^u, b+, b-, t+, t-] in nf=4,
### [T_3^u, V_3^u, T_8^d, V_8^d, t+, t-] in nf=5,
### [T_3^u, V_3^u, T_8^d, V_8^d, T_8^u, V_8^u] in nf=6

In [6]:
P = {}
ns, s, qed, qcd = "ns", "s", "qed", "qcd"
P[ns, qcd] = sympy.Array.zeros(14,14).as_mutable()
P[ns, qed] = sympy.Array.zeros(14,14).as_mutable()
P[s, qcd] =  sympy.Array.zeros(14,14).as_mutable()
P[s, qed] =  sympy.Array.zeros(14,14).as_mutable()

ei2=[eu2, ed2, ed2, eu2, ed2, eu2]
def es2_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return nu*eu2 + nd*ed2

def P_qcd(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=Pgg
    for i in range(1, nf+1):
        res[0, 2*i] = Pgq #g q+
        res[2*i, 0] = 2 * Pqg #q+ g
        res[2*i,2*i] = Pp #q+ q+
        res[1 + 2*i,1 + 2*i] = Pm #q- q-
    return res

def P_qed(nf):
    es2=es2_(nf)
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=es2*Pxgg
    res[1, 1]=es2*Pxyy
    res[0, 1]=es2*Pxgy
    res[1, 0]=es2*Pxyg
    for i in range(1, nf+1):
        res[0, 2*i] = ei2[i-1]*Pxgq
        res[2*i, 0] = 2*ei2[i-1]*Pxqg
        res[1, 2*i] = ei2[i-1]*Pxyq
        res[2*i, 1] = 2*ei2[i-1]*Pxqy
        res[2*i,2*i] = ei2[i-1]*Pxp 
        res[1 + 2*i,1 + 2*i] = ei2[i-1]*Pxm
    return res

def Ps_qcd(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    for i in range(1, nf+1):
        res[2*i, 2] = Pqq - Pp
        res[2*i, 3] = Pqq - Pp
        res[1 + 2*i, 4] = Pv - Pm
        res[1 + 2*i, 5] = Pv - Pm
    return res/nf

def Ps_qed(nf):
    res = sympy.Matrix.zeros(14,14).as_mutable()
    for i in range(1, nf+1):
        res[2*i, 2] = ei2[i-1]*eu2*(Pxqq - Pxp)
        res[2*i, 3] = ei2[i-1]*ed2*(Pxqq - Pxp)
    return res/nf

def P_uni(nf):
    return P_qcd(nf)+P_qed(nf)

def Ps_uni(nf):
    return Ps_qcd(nf)+Ps_qed(nf)

In [7]:
def rot_fl_to_ev(nf):
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=1
    res[1, 1]=1
    for i in range(2 + 2*nf, 14): 
        res[i,i] = 1
    for i in range(1,nf+1): #Sigma and V
        res[2, 2*i] = 1
        res[4, 1 + 2*i] = 1
    for i in [1, 4, 6]:#loop on up quarks
        if i <= nf:
            res[3, 2*i] = nd/nu
            res[5,1 + 2*i] = nd/nu
    for i in [2, 3, 5]:#loop on down quarks
        if i <= nf:
            res[3, 2*i] = -1
            res[5, 1 + 2*i] = -1
    if nf >= 3 :
        res[6, 4] = 1
        res[6, 6] = -1
        res[7, 5] = 1
        res[7, 7] = -1
    if nf >= 4 :
        res[8, 2] = 1
        res[8, 8] = -1
        res[9, 3] = 1
        res[9, 9] = -1
    if nf >= 5 :
        res[10, 4] = 1
        res[10, 6] = 1
        res[10, 10] = -2
        res[11, 5] = 1
        res[11, 7] = 1
        res[11, 11] = -2
    if nf == 6 :
        res[12, 2] = 1
        res[12, 8] = 1
        res[12, 12] = -2
        res[13, 3] = 1
        res[13, 9] = 1
        res[13, 13] = -2
    return res

def rot_ev_to_fl(nf):
    return rot_fl_to_ev(nf).inv()

In [8]:
def rot_sin_to_ev(nf):
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix.zeros(14,14).as_mutable()
    res[0, 0]=1
    res[1, 1]=1
    res[2,2]=1
    res[2,3]=1
    res[3,2]=nd/nu
    res[3,3]=-1
    res[4,4]=1
    res[4,5]=1
    res[5,4]=nd/nu
    res[5,5]=-1
    for i in range(6,14):
        res[i,i]=1
    return res

def rot_ev_to_sin(nf):
    return rot_sin_to_ev(nf).inv()

In [9]:
def P_ev(nf):
    res = rot_fl_to_ev(nf) * P_uni(nf) * rot_ev_to_fl(nf) + rot_fl_to_ev(nf) * Ps_uni(nf) * rot_ev_to_sin(nf)
    return res

In [10]:
P_ev(2)

Matrix([
[           P^x_gg*(e_d^2 + e_u^2) + P_gg,                  P^x_g\gamma*(e_d^2 + e_u^2),                                                                                                       0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                                -0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2,                                           0,                                           0, 0, 0, 0, 0, 0, 0, 0, 0],
[            P^x_\gamma g*(e_d^2 + e_u^2),             P^x_\gamma\gamma*(e_d^2 + e_u^2),                                                                                                      0.5*P^x_\gamma q*e_d^2 + 0.5*P^x_\gamma q*e_u^2,                                                                                                    -0.5*P^x_\gamma q*e_d^2 + 0.5*P^x_\gamma q*e_u^2,                                           0,                                           0, 0, 0, 0, 0

In [11]:
P_ev(3)

Matrix([
[         P^x_gg*(2*e_d^2 + e_u^2) + P_gg,                P^x_g\gamma*(2*e_d^2 + e_u^2),                                                                                                                                               0.666666666666667*P^x_gq*e_d^2 + 0.333333333333333*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                                                                        -0.333333333333333*P^x_gq*e_d^2 + 0.333333333333333*P^x_gq*e_u^2,                                                                       0,                                                                       0,                         0,                         0, 0, 0, 0, 0, 0, 0],
[          P^x_\gamma g*(2*e_d^2 + e_u^2),           P^x_\gamma\gamma*(2*e_d^2 + e_u^2),                                                                                                                                              0.666666666666667*

In [320]:
P_ev(4)

Matrix([
[       P^x_gg*(2*e_d^2 + 2*e_u^2) + P_gg,              P^x_g\gamma*(2*e_d^2 + 2*e_u^2),                                                                                                       0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                                -0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2,                                           0,                                           0,                         0,                         0,                         0,                         0, 0, 0, 0, 0],
[        P^x_\gamma g*(2*e_d^2 + 2*e_u^2),         P^x_\gamma\gamma*(2*e_d^2 + 2*e_u^2),                                                                                                      0.5*P^x_\gamma q*e_d^2 + 0.5*P^x_\gamma q*e_u^2,                                                                                                    -0.5*P^x_\gamma q*e_d^2 + 0.5*P^x_\gamma q*e_u^2,     

In [321]:
P_ev(5)

Matrix([
[        P^x_gg*(3*e_d^2 + 2*e_u^2) + P_gg,              P^x_g\gamma*(3*e_d^2 + 2*e_u^2),                                                                                                                          0.6*P^x_gq*e_d^2 + 0.4*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                       -0.4*P^x_gq*e_d^2 + 0.4*P^x_gq*e_u^2 + 2.77555756156289e-17*P_gq,                                                             0,                                                             0,                         0,                         0,                         0,                         0,                         0,                         0, 0, 0],
[         P^x_\gamma g*(3*e_d^2 + 2*e_u^2),         P^x_\gamma\gamma*(3*e_d^2 + 2*e_u^2),                                                                                                                         0.6*P^x_\gamma q*e_d^2 + 0.4*P^x_\gamma q*e_u^2,           

In [322]:
P_ev(6)

Matrix([
[        P^x_gg*(3*e_d^2 + 3*e_u^2) + P_gg,              P^x_g\gamma*(3*e_d^2 + 3*e_u^2),                                                                                                                                  0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2 + 1.0*P_gq,                                                                                                                -0.5*P^x_gq*e_d^2 + 0.5*P^x_gq*e_u^2,                                                                      0,                                           0,                         0,                         0,                         0,                         0,                         0,                         0,                         0,                         0],
[         P^x_\gamma g*(3*e_d^2 + 3*e_u^2),         P^x_\gamma\gamma*(3*e_d^2 + 3*e_u^2),                                                                                                                                 0.5*P^x_\gamma q*e_d^2 

In [323]:
def eD2_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return nd*eu2 + nu*ed2
def etam_(nf):
    nu = int(nf/2)
    nd = nf - nu
    return 0.5*(eu2 - ed2)

In [324]:
def P_ev_sing(nf):
    return P_ev(nf)[:4,:4]

def P_ev_val(nf):
    return P_ev(nf)[4:6,4:6]

In [1]:
def P_ev_sing2(nf):
    es2=es2_(nf)
    eD2=eD2_(nf)
    etam=etam_(nf)
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix([
        [Pgg + es2 * Pxgg, es2 * Pxgy, Pgq + es2/nf*Pxgq, 2*nu/nf*etam*Pxgq],
        [es2 * Pxyg, es2 * Pxyy, es2/nf*Pxyq, 2*nu/nf*etam*Pxyq],
        [2*nf*Pqg + 2*es2*Pxqg, 2*es2 * Pxqy, Pqq + es2/nf*Pxp +(es2/nf)**2*(Pxqq - Pxp), 2*nu/nf*etam*Pxp +2*nu*etam*es2/nf**2*(Pxqq - Pxp)],
        [4*nd*etam*Pxqg, 4*nd*etam*Pxqy, 2*nd/nf*etam*Pxp +2*nd*etam*es2/nf**2*(Pxqq - Pxp), Pp + eD2/nf*Pxp + 4*nu*nd/nf**2*etam**2*(Pxqq - Pxp)]
    ])
    return res

def P_ev_val2(nf):
    es2=es2_(nf)
    eD2=eD2_(nf)
    etam=etam_(nf)
    nu = int(nf/2)
    nd = nf - nu
    res = sympy.Matrix([
        [Pv+es2/nf*Pxm, 2*nu/nf*etam*Pxm],
        [2*nd/nf*etam*Pxm, Pm + eD2/nf*Pxm]
    ])
    return res

In [2]:
sympy.simplify(P_ev_sing(2)-P_ev_sing2(2))

NameError: name 'sympy' is not defined

In [327]:
sympy.simplify(P_ev_sing(3)-P_ev_sing2(3))

Matrix([
[0, 0, 0,                         0],
[0, 0, 0,                         0],
[0, 0, 0,                         0],
[0, 0, 0, -1.11022302462516e-16*P_+]])

In [328]:
sympy.simplify(P_ev_sing(4)-P_ev_sing2(4))

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [329]:
sympy.simplify(P_ev_sing(5)-P_ev_sing2(5))

Matrix([
[0, 0,                                                                                                                                                                                                                                8.88178419700125e-17*P^x_gq*e_d^2,                                                                                                                                                                2.77555756156289e-17*P_gq],
[0, 0,                                                                                                                                                                                                                          8.88178419700125e-17*P^x_\gamma q*e_d^2,                                                                                                                                                                                        0],
[0, 0,                                                                                 

In [330]:
sympy.simplify(P_ev_sing(6)-P_ev_sing2(6))

Matrix([
[0, 0, -1.11022302462516e-16*P_gq,                         0],
[0, 0,                          0,                         0],
[0, 0,  -1.11022302462516e-16*P_+,                         0],
[0, 0,                          0, -1.11022302462516e-16*P_+]])

In [331]:
sympy.simplify(P_ev_val(2)-P_ev_val2(2))

Matrix([
[0, 0],
[0, 0]])

In [332]:
sympy.simplify(P_ev_val(3)-P_ev_val2(3))

Matrix([
[0,                         0],
[0, -1.11022302462516e-16*P_-]])

In [333]:
sympy.simplify(P_ev_val(4)-P_ev_val2(4))

Matrix([
[0, 0],
[0, 0]])

In [334]:
sympy.simplify(P_ev_val(5)-P_ev_val2(5))

Matrix([
[                                                               8.88178419700125e-17*P^x_-*e_d^2,         2.77555756156289e-17*P_-],
[-1.11022302462516e-16*P^x_-*e_d^2 + 1.11022302462516e-16*P^x_-*e_u^2 + 5.55111512312578e-17*P_V, 8.88178419700125e-17*P^x_-*e_u^2]])

In [335]:
sympy.simplify(P_ev_val(6)-P_ev_val2(6))

Matrix([
[-1.11022302462516e-16*P_-,                         0],
[                        0, -1.11022302462516e-16*P_-]])